<a href="https://colab.research.google.com/github/sjkchang/CMPE255-Assignment-2/blob/master/notebooks/Assignment2_AnomalyDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pycaret
!pip install pycaret[analysis]


In [2]:
# check installed version
import pycaret
pycaret.__version__

'3.0.4'

In [3]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab Notebooks/Datasets/255-Datasets
%ls

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks/Datasets/255-Datasets
 3D-Print-Anomoly-Detection/
 Amazon-stock-time-series.csv
 CarPrice_Assignment.csv
'Credit Card Customer Data - Clustering.csv'
 falling-elders-anomoly-detection/
 flagged/
 Groceries_dataset-association-rules-mining.csv
 heartattack.csv
 logs.log
 my_first_pipeline.pkl
 secondhand-car-price-regression.csv


In [4]:
%cd falling-elders-anomoly-detection/
%ls

/content/drive/MyDrive/Colab Notebooks/Datasets/255-Datasets/falling-elders-anomoly-detection
test/  train/


In [5]:
%cd train/
%ls

/content/drive/MyDrive/Colab Notebooks/Datasets/255-Datasets/falling-elders-anomoly-detection/train
data_0.csv   data_13.csv  data_17.csv  data_2.csv  data_6.csv  logs.log
data_10.csv  data_14.csv  data_18.csv  data_3.csv  data_7.csv
data_11.csv  data_15.csv  data_19.csv  data_4.csv  data_8.csv
data_12.csv  data_16.csv  data_1.csv   data_5.csv  data_9.csv


In [6]:
# loading sample dataset from pycaret dataset module
from pycaret.datasets import get_data
train_data_0 = get_data('data_0')
train_data_1 = get_data('data_1')
train_data_2 = get_data('data_2')
train_data_3 = get_data('data_3')
train_data_4 = get_data('data_4')
train_data_5 = get_data('data_5')
train_data_6 = get_data('data_6')
train_data_7 = get_data('data_7')
train_data_8 = get_data('data_8')
train_data_9 = get_data('data_9')



,x,y,z,010-000-024-033,010-000-030-096,020-000-032-221,020-000-033-111,anomaly
0,18.495860,13.766527,14.362624,0.0,0.0,0.0,1.0,0.0
1,18.501072,13.827225,14.270268,0.0,0.0,1.0,0.0,0.0
2,18.405950,13.868976,14.094804,1.0,0.0,0.0,0.0,0.0
3,18.444572,13.910701,14.116078,0.0,1.0,0.0,0.0,0.0
4,18.418470,13.933917,14.320566,0.0,0.0,0.0,1.0,0.0


,x,y,z,010-000-024-033,010-000-030-096,020-000-032-221,020-000-033-111,anomaly
0,19.298230,15.044445,13.195140,0.0,0.0,0.0,1.0,0.0
1,19.273322,14.992369,13.298404,0.0,0.0,1.0,0.0,0.0
2,19.392364,15.059970,13.320235,1.0,0.0,0.0,0.0,0.0
3,19.455964,15.221361,13.124152,0.0,1.0,0.0,0.0,0.0
4,18.499571,15.299723,13.280978,0.0,0.0,0.0,1.0,0.0


,x,y,z,010-000-024-033,010-000-030-096,020-000-032-221,020-000-033-111,anomaly
0,24.360715,14.661491,15.061292,0.0,0.0,0.0,1.0,0.0
1,24.345720,14.677384,15.080112,0.0,0.0,1.0,0.0,0.0
2,24.315322,14.641433,15.020421,1.0,0.0,0.0,0.0,0.0
3,24.305469,14.663497,15.005379,0.0,1.0,0.0,0.0,0.0
4,24.273169,14.633127,14.865332,0.0,0.0,1.0,0.0,0.0


,x,y,z,010-000-024-033,010-000-030-096,020-000-032-221,020-000-033-111,anomaly
0,4.358617,12.052514,10.516318,0.0,1.0,0.0,0.0,0.0
1,4.372627,12.059844,10.511330,0.0,0.0,1.0,0.0,0.0
2,4.369362,12.078273,10.140976,0.0,1.0,0.0,0.0,0.0
3,4.386196,11.957048,10.042876,1.0,0.0,0.0,0.0,0.0
4,4.394224,11.929170,10.347901,0.0,0.0,1.0,0.0,0.0


,x,y,z,010-000-024-033,010-000-030-096,020-000-032-221,020-000-033-111,anomaly
0,24.272887,13.254991,13.789312,0.0,0.0,1.0,0.0,0.0
1,24.234011,13.150937,13.929036,0.0,1.0,0.0,0.0,0.0
2,24.208880,13.240454,14.058172,0.0,0.0,0.0,1.0,0.0
3,24.182305,13.195297,14.082580,1.0,0.0,0.0,0.0,0.0
4,24.156595,13.240984,14.096994,0.0,0.0,1.0,0.0,0.0


,x,y,z,010-000-024-033,010-000-030-096,020-000-032-221,020-000-033-111,anomaly
0,22.611324,14.994165,11.863249,0.0,1.0,0.0,0.0,0.0
1,22.660755,15.002137,12.237482,0.0,0.0,0.0,1.0,0.0
2,22.566564,15.109436,12.273901,1.0,0.0,0.0,0.0,0.0
3,22.595878,15.060713,12.087886,0.0,1.0,0.0,0.0,0.0
4,22.555830,15.025164,11.958934,0.0,0.0,1.0,0.0,0.0


,x,y,z,010-000-024-033,010-000-030-096,020-000-032-221,020-000-033-111,anomaly
0,5.291177,8.247071,10.318103,0.0,0.0,0.0,1.0,0.0
1,5.282482,8.213895,10.047694,1.0,0.0,0.0,0.0,0.0
2,5.305478,8.273616,9.703028,0.0,1.0,0.0,0.0,0.0
3,5.275765,8.289640,9.899360,0.0,0.0,1.0,0.0,0.0
4,5.245949,8.225723,10.228732,0.0,0.0,0.0,1.0,0.0


,x,y,z,010-000-024-033,010-000-030-096,020-000-032-221,020-000-033-111,anomaly
0,5.285550,10.083548,13.587454,0.0,0.0,1.0,0.0,0.0
1,5.287776,10.047991,13.901551,0.0,0.0,0.0,1.0,0.0
2,5.365385,10.150613,13.499768,1.0,0.0,0.0,0.0,0.0
3,5.327014,10.246569,13.762199,0.0,0.0,1.0,0.0,0.0
4,5.310819,10.173506,13.550943,0.0,1.0,0.0,0.0,0.0


,x,y,z,010-000-024-033,010-000-030-096,020-000-032-221,020-000-033-111,anomaly
0,23.319128,14.510240,12.478010,0.0,1.0,0.0,0.0,0.0
1,23.289727,14.508383,12.323765,1.0,0.0,0.0,0.0,0.0
2,23.310721,14.361386,12.517157,0.0,0.0,1.0,0.0,0.0
3,23.342259,14.511974,12.521861,0.0,1.0,0.0,0.0,0.0
4,23.301868,14.496082,12.262493,1.0,0.0,0.0,0.0,0.0


,x,y,z,010-000-024-033,010-000-030-096,020-000-032-221,020-000-033-111,anomaly
0,6.325685,10.449807,9.480258,0.0,0.0,1.0,0.0,0.0
1,6.342243,10.328274,9.486351,0.0,1.0,0.0,0.0,0.0
2,6.381572,10.477760,9.514933,1.0,0.0,0.0,0.0,0.0
3,6.327767,10.289277,9.736243,0.0,0.0,1.0,0.0,0.0
4,6.358251,10.353780,9.387763,0.0,1.0,0.0,0.0,0.0


In [7]:
%cd ../test/
%ls

/content/drive/MyDrive/Colab Notebooks/Datasets/255-Datasets/falling-elders-anomoly-detection/test
data_20.csv  data_21.csv  data_22.csv  data_23.csv  data_24.csv  logs.log


In [8]:
test_data_20 = get_data('data_20')
test_data_21 = get_data('data_21')
test_data_22 = get_data('data_22')
test_data_23 = get_data('data_23')
test_data_24 = get_data('data_24')


,x,y,z,010-000-024-033,010-000-030-096,020-000-032-221,020-000-033-111,anomaly
0,6.912997,11.518698,15.471855,0.0,0.0,0.0,1.0,0.0
1,6.936432,11.574586,15.446939,0.0,0.0,1.0,0.0,0.0
2,6.935274,11.571790,15.437505,1.0,0.0,0.0,0.0,0.0
3,6.886688,11.561593,15.704019,0.0,0.0,0.0,1.0,0.0
4,6.921823,11.597728,15.634435,0.0,0.0,1.0,0.0,0.0


,x,y,z,010-000-024-033,010-000-030-096,020-000-032-221,020-000-033-111,anomaly
0,16.133636,18.036322,11.785302,0.0,0.0,1.0,0.0,0.0
1,16.244710,18.161550,11.781509,1.0,0.0,0.0,0.0,0.0
2,16.220005,18.090391,11.342841,0.0,1.0,0.0,0.0,0.0
3,16.200422,18.035811,11.457980,0.0,0.0,0.0,1.0,0.0
4,16.165606,18.240522,11.694980,0.0,0.0,1.0,0.0,0.0


,x,y,z,010-000-024-033,010-000-030-096,020-000-032-221,020-000-033-111,anomaly
0,15.070527,15.572755,11.506120,0.0,1.0,0.0,0.0,0.0
1,15.018874,15.579306,11.493767,0.0,0.0,0.0,1.0,0.0
2,15.014778,15.571672,11.490566,1.0,0.0,0.0,0.0,0.0
3,15.025130,15.641097,11.516335,0.0,0.0,1.0,0.0,0.0
4,15.046580,15.580838,11.526773,0.0,1.0,0.0,0.0,0.0


,x,y,z,010-000-024-033,010-000-030-096,020-000-032-221,020-000-033-111,anomaly
0,13.201092,11.066999,14.010642,0.0,0.0,1.0,0.0,0.0
1,13.179607,11.026071,13.796169,1.0,0.0,0.0,0.0,0.0
2,13.121556,10.928069,14.057485,0.0,0.0,0.0,1.0,0.0
3,13.190235,11.065608,13.984199,0.0,1.0,0.0,0.0,0.0
4,13.191192,11.066251,13.761203,1.0,0.0,0.0,0.0,0.0


,x,y,z,010-000-024-033,010-000-030-096,020-000-032-221,020-000-033-111,anomaly
0,6.504078,12.789729,9.447159,1.0,0.0,0.0,0.0,0.0
1,6.472978,12.675193,9.622985,0.0,0.0,1.0,0.0,0.0
2,6.452725,12.625970,9.325761,0.0,1.0,0.0,0.0,0.0
3,6.426322,12.610741,9.753680,0.0,0.0,0.0,1.0,0.0
4,6.401717,12.602613,9.780900,1.0,0.0,0.0,0.0,0.0


In [9]:
# import pycaret anomaly and init setup
from pycaret.anomaly import *
s = setup(train_data_0, session_id = 123)

,Description,Value
0,Session id,123
1,Original data shape,"(5805, 8)"
2,Transformed data shape,"(5805, 8)"
3,Numeric features,8
4,Preprocess,True
5,Imputation type,simple
6,Numeric imputation,mean
7,Categorical imputation,mode
8,CPU Jobs,-1
9,Use GPU,False


In [10]:
# train iforest model
iforest = create_model('iforest')
iforest

Processing:   0%|          | 0/3 [00:00<?, ?it/s]

IForest(behaviour='new', bootstrap=False, contamination=0.05,
    max_features=1.0, max_samples='auto', n_estimators=100, n_jobs=-1,
    random_state=123, verbose=0)

In [17]:
iforest_anomalies = assign_model(iforest)
iforest_anomalies

,x,y,z,010-000-024-033,010-000-030-096,020-000-032-221,020-000-033-111,anomaly,Anomaly,Anomaly_Score
0,18.495859,13.766527,14.362624,0.0,0.0,0.0,1.0,0.0,0,-0.029032
1,18.501072,13.827225,14.270267,0.0,0.0,1.0,0.0,0.0,0,-0.030058
2,18.405951,13.868976,14.094804,1.0,0.0,0.0,0.0,0.0,0,-0.036673
3,18.444572,13.910701,14.116078,0.0,1.0,0.0,0.0,0.0,1,0.006197
4,18.418470,13.933917,14.320566,0.0,0.0,0.0,1.0,0.0,0,-0.031068
...,...,...,...,...,...,...,...,...,...,...
5800,13.071400,13.964593,14.014944,0.0,0.0,0.0,1.0,0.0,0,-0.091531
5801,13.170664,14.089759,13.972008,0.0,0.0,1.0,0.0,0.0,0,-0.099864
5802,13.262425,14.129938,13.811263,1.0,0.0,0.0,0.0,0.0,0,-0.102784
5803,13.282711,14.161534,13.781787,0.0,1.0,0.0,0.0,0.0,0,-0.051041


In [21]:
# predict on test set
iforest_pred = predict_model(iforest, data=test_data_20)
iforest_pred

,x,y,z,010-000-024-033,010-000-030-096,020-000-032-221,020-000-033-111,anomaly,Anomaly,Anomaly_Score
0,6.912997,11.518698,15.471855,0.0,0.0,0.0,1.0,0.0,0,-0.036738
1,6.936432,11.574586,15.446939,0.0,0.0,1.0,0.0,0.0,0,-0.024183
2,6.935274,11.571790,15.437505,1.0,0.0,0.0,0.0,0.0,0,-0.036551
3,6.886688,11.561593,15.704019,0.0,0.0,0.0,1.0,0.0,0,-0.036738
4,6.921823,11.597728,15.634435,0.0,0.0,1.0,0.0,0.0,0,-0.025884
...,...,...,...,...,...,...,...,...,...,...
6618,7.619840,11.961406,15.810897,1.0,0.0,0.0,0.0,0.0,0,-0.046839
6619,7.601608,11.877016,16.006231,0.0,0.0,0.0,1.0,0.0,0,-0.045467
6620,7.633920,11.892471,16.117411,0.0,0.0,1.0,0.0,0.0,0,-0.037572
6621,7.655626,12.022767,15.907901,1.0,0.0,0.0,0.0,0.0,0,-0.047420


import numpy as np
from google.colab import autoviz

def value_plot(df, y, figscale=1):
  from matplotlib import pyplot as plt
  df[y].plot(kind='line', figsize=(8 * figscale, 4 * figscale), title=y)
  plt.gca().spines[['top', 'right']].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(iforest_pred, *['x'], **{})
chart

import numpy as np
from google.colab import autoviz

def value_plot(df, y, figscale=1):
  from matplotlib import pyplot as plt
  df[y].plot(kind='line', figsize=(8 * figscale, 4 * figscale), title=y)
  plt.gca().spines[['top', 'right']].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(iforest_pred, *['y'], **{})
chart

import numpy as np
from google.colab import autoviz

def value_plot(df, y, figscale=1):
  from matplotlib import pyplot as plt
  df[y].plot(kind='line', figsize=(8 * figscale, 4 * figscale), title=y)
  plt.gca().spines[['top', 'right']].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(iforest_pred, *['z'], **{})
chart

import numpy as np
from google.colab import autoviz

def value_plot(df, y, figscale=1):
  from matplotlib import pyplot as plt
  df[y].plot(kind='line', figsize=(8 * figscale, 4 * figscale), title=y)
  plt.gca().spines[['top', 'right']].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(iforest_pred, *['010-000-024-033'], **{})
chart

import numpy as np
from google.colab import autoviz

def histogram(df, colname, num_bins=20, figscale=1):
  from matplotlib import pyplot as plt
  df[colname].plot(kind='hist', bins=num_bins, title=colname, figsize=(8*figscale, 4*figscale))
  plt.gca().spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = histogram(iforest_pred, *['x'], **{})
chart

import numpy as np
from google.colab import autoviz

def histogram(df, colname, num_bins=20, figscale=1):
  from matplotlib import pyplot as plt
  df[colname].plot(kind='hist', bins=num_bins, title=colname, figsize=(8*figscale, 4*figscale))
  plt.gca().spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = histogram(iforest_pred, *['y'], **{})
chart

import numpy as np
from google.colab import autoviz

def histogram(df, colname, num_bins=20, figscale=1):
  from matplotlib import pyplot as plt
  df[colname].plot(kind='hist', bins=num_bins, title=colname, figsize=(8*figscale, 4*figscale))
  plt.gca().spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = histogram(iforest_pred, *['z'], **{})
chart

import numpy as np
from google.colab import autoviz

def histogram(df, colname, num_bins=20, figscale=1):
  from matplotlib import pyplot as plt
  df[colname].plot(kind='hist', bins=num_bins, title=colname, figsize=(8*figscale, 4*figscale))
  plt.gca().spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = histogram(iforest_pred, *['010-000-024-033'], **{})
chart

import numpy as np
from google.colab import autoviz

def scatter_plots(df, colname_pairs, figscale=1, alpha=.8):
  from matplotlib import pyplot as plt
  plt.figure(figsize=(len(colname_pairs) * 6 * figscale, 6 * figscale))
  for plot_i, (x_colname, y_colname) in enumerate(colname_pairs, start=1):
    ax = plt.subplot(1, len(colname_pairs), plot_i)
    df.plot(kind='scatter', x=x_colname, y=y_colname, s=(32 * figscale), alpha=alpha, ax=ax)
    ax.spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = scatter_plots(iforest_pred, *[[['x', 'y'], ['y', 'z'], ['z', '010-000-024-033'], ['010-000-024-033', '010-000-030-096']]], **{})
chart

In [23]:
# train KNN model
knn = create_model('knn')
knn

Processing:   0%|          | 0/3 [00:00<?, ?it/s]

KNN(algorithm='auto', contamination=0.05, leaf_size=30, method='largest',
  metric='minkowski', metric_params=None, n_jobs=-1, n_neighbors=5, p=2,
  radius=1.0)

In [18]:
knn = assign_model(knn)
knn

,x,y,z,010-000-024-033,010-000-030-096,020-000-032-221,020-000-033-111,anomaly,Anomaly,Anomaly_Score
0,18.495859,13.766527,14.362624,0.0,0.0,0.0,1.0,0.0,0,0.296666
1,18.501072,13.827225,14.270267,0.0,0.0,1.0,0.0,0.0,0,0.389512
2,18.405951,13.868976,14.094804,1.0,0.0,0.0,0.0,0.0,0,0.347593
3,18.444572,13.910701,14.116078,0.0,1.0,0.0,0.0,0.0,0,0.499612
4,18.418470,13.933917,14.320566,0.0,0.0,0.0,1.0,0.0,0,0.261788
...,...,...,...,...,...,...,...,...,...,...
5800,13.071400,13.964593,14.014944,0.0,0.0,0.0,1.0,0.0,1,1.033235
5801,13.170664,14.089759,13.972008,0.0,0.0,1.0,0.0,0.0,1,0.929784
5802,13.262425,14.129938,13.811263,1.0,0.0,0.0,0.0,0.0,1,0.810826
5803,13.282711,14.161534,13.781787,0.0,1.0,0.0,0.0,0.0,1,0.952925


In [24]:
# predict on test set
knn_pred = predict_model(knn, data=test_data_20)
knn_pred

,x,y,z,010-000-024-033,010-000-030-096,020-000-032-221,020-000-033-111,anomaly,Anomaly,Anomaly_Score
0,6.912997,11.518698,15.471855,0.0,0.0,0.0,1.0,0.0,1,2.364893
1,6.936432,11.574586,15.446939,0.0,0.0,1.0,0.0,0.0,1,2.182945
2,6.935274,11.571790,15.437505,1.0,0.0,0.0,0.0,0.0,1,2.316116
3,6.886688,11.561593,15.704019,0.0,0.0,0.0,1.0,0.0,1,2.501617
4,6.921823,11.597728,15.634435,0.0,0.0,1.0,0.0,0.0,1,2.312727
...,...,...,...,...,...,...,...,...,...,...
6618,7.619840,11.961406,15.810897,1.0,0.0,0.0,0.0,0.0,1,2.281590
6619,7.601608,11.877016,16.006231,0.0,0.0,0.0,1.0,0.0,1,2.451089
6620,7.633920,11.892471,16.117411,0.0,0.0,1.0,0.0,0.0,1,2.459253
6621,7.655626,12.022767,15.907901,1.0,0.0,0.0,0.0,0.0,1,2.340631


import numpy as np
from google.colab import autoviz

def value_plot(df, y, figscale=1):
  from matplotlib import pyplot as plt
  df[y].plot(kind='line', figsize=(8 * figscale, 4 * figscale), title=y)
  plt.gca().spines[['top', 'right']].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(knn_pred, *['x'], **{})
chart

import numpy as np
from google.colab import autoviz

def value_plot(df, y, figscale=1):
  from matplotlib import pyplot as plt
  df[y].plot(kind='line', figsize=(8 * figscale, 4 * figscale), title=y)
  plt.gca().spines[['top', 'right']].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(knn_pred, *['y'], **{})
chart

import numpy as np
from google.colab import autoviz

def value_plot(df, y, figscale=1):
  from matplotlib import pyplot as plt
  df[y].plot(kind='line', figsize=(8 * figscale, 4 * figscale), title=y)
  plt.gca().spines[['top', 'right']].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(knn_pred, *['z'], **{})
chart

import numpy as np
from google.colab import autoviz

def value_plot(df, y, figscale=1):
  from matplotlib import pyplot as plt
  df[y].plot(kind='line', figsize=(8 * figscale, 4 * figscale), title=y)
  plt.gca().spines[['top', 'right']].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(knn_pred, *['010-000-024-033'], **{})
chart

import numpy as np
from google.colab import autoviz

def histogram(df, colname, num_bins=20, figscale=1):
  from matplotlib import pyplot as plt
  df[colname].plot(kind='hist', bins=num_bins, title=colname, figsize=(8*figscale, 4*figscale))
  plt.gca().spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = histogram(knn_pred, *['x'], **{})
chart

import numpy as np
from google.colab import autoviz

def histogram(df, colname, num_bins=20, figscale=1):
  from matplotlib import pyplot as plt
  df[colname].plot(kind='hist', bins=num_bins, title=colname, figsize=(8*figscale, 4*figscale))
  plt.gca().spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = histogram(knn_pred, *['y'], **{})
chart

import numpy as np
from google.colab import autoviz

def histogram(df, colname, num_bins=20, figscale=1):
  from matplotlib import pyplot as plt
  df[colname].plot(kind='hist', bins=num_bins, title=colname, figsize=(8*figscale, 4*figscale))
  plt.gca().spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = histogram(knn_pred, *['z'], **{})
chart

import numpy as np
from google.colab import autoviz

def histogram(df, colname, num_bins=20, figscale=1):
  from matplotlib import pyplot as plt
  df[colname].plot(kind='hist', bins=num_bins, title=colname, figsize=(8*figscale, 4*figscale))
  plt.gca().spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = histogram(knn_pred, *['010-000-024-033'], **{})
chart

import numpy as np
from google.colab import autoviz

def scatter_plots(df, colname_pairs, figscale=1, alpha=.8):
  from matplotlib import pyplot as plt
  plt.figure(figsize=(len(colname_pairs) * 6 * figscale, 6 * figscale))
  for plot_i, (x_colname, y_colname) in enumerate(colname_pairs, start=1):
    ax = plt.subplot(1, len(colname_pairs), plot_i)
    df.plot(kind='scatter', x=x_colname, y=y_colname, s=(32 * figscale), alpha=alpha, ax=ax)
    ax.spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = scatter_plots(knn_pred, *[[['x', 'y'], ['y', 'z'], ['z', '010-000-024-033'], ['010-000-024-033', '010-000-030-096']]], **{})
chart

In [25]:
# train SVM model
svm_model  = create_model('svm')
svm_model

Processing:   0%|          | 0/3 [00:00<?, ?it/s]

OCSVM(cache_size=200, coef0=0.0, contamination=0.05, degree=3, gamma='auto',
   kernel='rbf', max_iter=-1, nu=0.5, shrinking=True, tol=0.001,
   verbose=False)

In [20]:
svm_model = assign_model(svm_model)
svm_model

,x,y,z,010-000-024-033,010-000-030-096,020-000-032-221,020-000-033-111,anomaly,Anomaly,Anomaly_Score
0,18.495859,13.766527,14.362624,0.0,0.0,0.0,1.0,0.0,1,203.374209
1,18.501072,13.827225,14.270267,0.0,0.0,1.0,0.0,0.0,1,202.516056
2,18.405951,13.868976,14.094804,1.0,0.0,0.0,0.0,0.0,1,195.079419
3,18.444572,13.910701,14.116078,0.0,1.0,0.0,0.0,0.0,1,203.155012
4,18.418470,13.933917,14.320566,0.0,0.0,0.0,1.0,0.0,1,201.999028
...,...,...,...,...,...,...,...,...,...,...
5800,13.071400,13.964593,14.014944,0.0,0.0,0.0,1.0,0.0,0,22.373459
5801,13.170664,14.089759,13.972008,0.0,0.0,1.0,0.0,0.0,0,16.211012
5802,13.262425,14.129938,13.811263,1.0,0.0,0.0,0.0,0.0,0,3.792348
5803,13.282711,14.161534,13.781787,0.0,1.0,0.0,0.0,0.0,0,10.076754


In [27]:
# predict on test set
svm_pred = predict_model(svm_model, data=test_data_20)
svm_pred

,x,y,z,010-000-024-033,010-000-030-096,020-000-032-221,020-000-033-111,anomaly,Anomaly,Anomaly_Score
0,6.912997,11.518698,15.471855,0.0,0.0,0.0,1.0,0.0,1,259.764116
1,6.936432,11.574586,15.446939,0.0,0.0,1.0,0.0,0.0,1,256.364033
2,6.935274,11.571790,15.437505,1.0,0.0,0.0,0.0,0.0,1,256.363698
3,6.886688,11.561593,15.704019,0.0,0.0,0.0,1.0,0.0,1,271.953816
4,6.921823,11.597728,15.634435,0.0,0.0,1.0,0.0,0.0,1,266.654820
...,...,...,...,...,...,...,...,...,...,...
6618,7.619840,11.961406,15.810897,1.0,0.0,0.0,0.0,0.0,1,253.976620
6619,7.601608,11.877016,16.006231,0.0,0.0,0.0,1.0,0.0,1,266.767769
6620,7.633920,11.892471,16.117411,0.0,0.0,1.0,0.0,0.0,1,271.755477
6621,7.655626,12.022767,15.907901,1.0,0.0,0.0,0.0,0.0,1,258.198398


import numpy as np
from google.colab import autoviz

def value_plot(df, y, figscale=1):
  from matplotlib import pyplot as plt
  df[y].plot(kind='line', figsize=(8 * figscale, 4 * figscale), title=y)
  plt.gca().spines[['top', 'right']].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(svm_pred, *['x'], **{})
chart

import numpy as np
from google.colab import autoviz

def value_plot(df, y, figscale=1):
  from matplotlib import pyplot as plt
  df[y].plot(kind='line', figsize=(8 * figscale, 4 * figscale), title=y)
  plt.gca().spines[['top', 'right']].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(svm_pred, *['y'], **{})
chart

import numpy as np
from google.colab import autoviz

def value_plot(df, y, figscale=1):
  from matplotlib import pyplot as plt
  df[y].plot(kind='line', figsize=(8 * figscale, 4 * figscale), title=y)
  plt.gca().spines[['top', 'right']].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(svm_pred, *['z'], **{})
chart

import numpy as np
from google.colab import autoviz

def value_plot(df, y, figscale=1):
  from matplotlib import pyplot as plt
  df[y].plot(kind='line', figsize=(8 * figscale, 4 * figscale), title=y)
  plt.gca().spines[['top', 'right']].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(svm_pred, *['010-000-024-033'], **{})
chart

import numpy as np
from google.colab import autoviz

def histogram(df, colname, num_bins=20, figscale=1):
  from matplotlib import pyplot as plt
  df[colname].plot(kind='hist', bins=num_bins, title=colname, figsize=(8*figscale, 4*figscale))
  plt.gca().spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = histogram(svm_pred, *['x'], **{})
chart

import numpy as np
from google.colab import autoviz

def histogram(df, colname, num_bins=20, figscale=1):
  from matplotlib import pyplot as plt
  df[colname].plot(kind='hist', bins=num_bins, title=colname, figsize=(8*figscale, 4*figscale))
  plt.gca().spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = histogram(svm_pred, *['y'], **{})
chart

import numpy as np
from google.colab import autoviz

def histogram(df, colname, num_bins=20, figscale=1):
  from matplotlib import pyplot as plt
  df[colname].plot(kind='hist', bins=num_bins, title=colname, figsize=(8*figscale, 4*figscale))
  plt.gca().spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = histogram(svm_pred, *['z'], **{})
chart

import numpy as np
from google.colab import autoviz

def histogram(df, colname, num_bins=20, figscale=1):
  from matplotlib import pyplot as plt
  df[colname].plot(kind='hist', bins=num_bins, title=colname, figsize=(8*figscale, 4*figscale))
  plt.gca().spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = histogram(svm_pred, *['010-000-024-033'], **{})
chart

import numpy as np
from google.colab import autoviz

def scatter_plots(df, colname_pairs, figscale=1, alpha=.8):
  from matplotlib import pyplot as plt
  plt.figure(figsize=(len(colname_pairs) * 6 * figscale, 6 * figscale))
  for plot_i, (x_colname, y_colname) in enumerate(colname_pairs, start=1):
    ax = plt.subplot(1, len(colname_pairs), plot_i)
    df.plot(kind='scatter', x=x_colname, y=y_colname, s=(32 * figscale), alpha=alpha, ax=ax)
    ax.spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = scatter_plots(svm_pred, *[[['x', 'y'], ['y', 'z'], ['z', '010-000-024-033'], ['010-000-024-033', '010-000-030-096']]], **{})
chart